### DWT and modelling

This notebook contains DWT(discrete wavelet transform) filtering technique and clustering perfomred over the filtered waveform.

In [ ]:
from standardize import *

In [ ]:
import pandas as pd

In [ ]:
udata = pd.read_csv("/Users/neethug/Desktop/Neethu/Course/DATA599/Project/finaldata/Windowed Time Series/un_cal.csv")
pdata = pd.read_csv("/Users/neethug/Desktop/Neethu/Course/DATA599/Project/finaldata/Windowed Time Series/ecd_cal.csv")

In [ ]:
pdata['Label'] = "ECD"

In [ ]:
udata['Label'] = "UnSuccessful"

In [ ]:
data = pd.concat([pdata,udata])

In [ ]:
data[data['TestId'] == 9610462]

In [ ]:
data[data['TestId'] == 9610647]

In [ ]:
data = data.drop([data.index[140] , data.index[141]])

In [ ]:
data = data.reset_index()

In [ ]:
data.drop('index', axis=1, inplace=True)

In [ ]:
bool_series = data['TestId'].duplicated()
bool_series.value_counts()

In [ ]:
A = data.iloc[:,1:-1]

### Discrete Fourier transform

In [ ]:
from scipy.fft import fft, ifft

In [ ]:
y = fft(A)

In [ ]:
df = pd. DataFrame(abs(y))

In [ ]:
df

In [ ]:
df['TestId'] = data['TestId']

In [ ]:
df['Label'] = data['Label']

In [ ]:
df

In [ ]:
A[0]

In [ ]:
### finding peaks

In [ ]:
from scipy.signal import find_peaks

In [ ]:
peaks = find_peaks(A[0], height=0)

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()

In [ ]:
alt.Chart(df).mark_circle(size=60).encode(
    x='DFT1',
    y='DFT2',
    color = 'Label',
    tooltip=['TestId']
)

## Fourier transform works when no variation in time happens, when we are in the frequency-domain we lose this dependency

In some way we can think of the Fourier transform as a trade-off between time information and frequency information. By taking a FT of a time signal, all time information is lost in return for frequency information

## Discrete Wavelet Transform

This considers both frequency and time aspects of the waveform ,
A discrete wavelet transform (DWT) is a transform that decomposes a given signal into a number of sets, where each set is a time series of coefficients describing the time evolution of the signal in the corresponding frequency band.

In [ ]:
import pywt

In [ ]:
res, res1 = pywt.dwt(A,wavelet='Haar')

In [ ]:
res

In [ ]:
len(res1)

In [ ]:
df2 = pd.DataFrame(res1)

In [ ]:
df2

In [ ]:
df2['TestId'] = data['TestId']

In [ ]:
df2['Label'] = data['Label']

In [ ]:
df2

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn import metrics

In [ ]:
X = df2.iloc[:,0:30]
Y = df2.iloc[:,-1]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle = True)

In [ ]:
res = RandomForestClassifier(n_estimators=100)
res.fit(X_train,Y_train)

In [ ]:
Y_pred=res.predict(X_train)

In [ ]:
print(metrics.confusion_matrix(Y_train, Y_pred))

In [ ]:
Y_pred1=res.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(Y_test, Y_pred1))

In [ ]:
print(classification_report(Y_test, Y_pred1))

#### we saw a single level decoimposition above now lets see a three level decomposition

In [ ]:
import matplotlib.pyplot as plt
data = A
waveletname = 'haar'
 
fig, axarr = plt.subplots(nrows=3, ncols=2, figsize=(6,6))
for ii in range(3):
    (data, coeff_d) = pywt.dwt(data, waveletname)
    axarr[ii, 0].plot(data, 'r')
    axarr[ii, 1].plot(coeff_d, 'g')
    axarr[ii, 0].set_ylabel("Level {}".format(ii + 1), fontsize=14, rotation=90)
    axarr[ii, 0].set_yticklabels([])
    if ii == 0:
        axarr[ii, 0].set_title("Approximation coefficients", fontsize=14)
        axarr[ii, 1].set_title("Detail coefficients", fontsize=14)
    axarr[ii, 1].set_yticklabels([])
plt.tight_layout()
plt.show()

In [ ]:
res, res2 = pywt.dwt(res,wavelet='Haar')

In [ ]:
res, res3 = pywt.dwt(res,wavelet='Haar')

In [ ]:
test = pd.DataFrame(res3)

In [ ]:
test

In [ ]:
test['TestId'] = df2['TestId']

In [ ]:
test['Label'] = df2['Label']

In [ ]:
test

In [ ]:
X1 = test.iloc[:,0:8]
Y1 = df2.iloc[:,-1]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X1, Y1, test_size=0.2, random_state=42, shuffle = True)

In [ ]:
mod = RandomForestClassifier(n_estimators=100)
mod.fit(X_train,Y_train)

In [ ]:
Y_pred=mod.predict(X_train)

In [ ]:
print(metrics.confusion_matrix(Y_train, Y_pred))

In [ ]:
Y_pred1=mod.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(Y_test, Y_pred1))

In [ ]:
print(classification_report(Y_test, Y_pred1))

## did we dilute the data???

#### Lets use other different clustering models to see

In [ ]:
##Affinity propagation

from sklearn.cluster import AffinityPropagation

In [ ]:
model = AffinityPropagation(damping=0.9)

In [ ]:
model.fit(X_train)

In [ ]:
yhat = model.predict(X_test)

In [ ]:
yhat